In [24]:
from features import ensembleFeature
import pandas as pd
import numpy as np
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold
from features import ensembleFeature

In [25]:
def file_va(filename):
    f = open(filename,"rb")
    m=0
    name = []
    seq = []
    struc = []
    for i in f:
        #print(i)
        m+=1
        if m>2 and (m-2)%3==1:
            name.append(str(i[1:-1],'utf-8'))
        if m>3 and (m-2)%3==2:
            seq.append(str(i[:-1],'utf-8'))
        if m>3 and (m-2)%3==0:
            struc.append(str(i,'utf-8')[:41])
    return name,seq,struc

def RNAfold_dot(filename):
    f = open(filename,"rb")
    m = 0
    struc = []
    seq = []
    for i in f:
        m+=1
        if m%6==3:
            struc.append(str(i[0:41],'utf-8'))
        if m%6==2:
            seq.append(str(i[0:41],'utf-8'))
    return seq,struc

def struc_01(data):
    x = np.zeros(len(data)*2).reshape(-1,2)
    for i in range(len(data)):
        if data[i]=="(":
            x[i][0]=1
        elif data[i]==")":
            x[i][1]=1
    return x

def struc_01_8(data):
    x = np.zeros((len(data)-2)*8).reshape(-1,8)
    for i in range(len(data)-2):
        mm = 0
        if data[i]=="(" or data[i]==")":
            mm+=4
        if data[i+1]=="(" or data[i]==")":
            mm+=2
        if data[i+2]=="(" or data[i]==")":
            mm+=1
        x[i][mm]=1
    return x

In [26]:
train_name,train_seq,train_struc = file_va("data/mouse_train.log")
trainPosLabel = np.ones(int(len(train_seq)/2))
trainNegLabel = np.zeros(int(len(train_seq)/2))
trainLabel = np.append(trainPosLabel,trainNegLabel)
#获取01二级结构表示
trainData = []
for i in range(len(train_struc)):
    trainData.append(struc_01(train_struc[i]))
trainData = np.array(trainData)
        
#与单热编码结合
trainData_origin = np.append(trainData,ensembleFeature.Binary(train_seq),axis=2).reshape(trainData.shape[0],-1)

In [4]:
import pymrmr
colu = [f'fea_{i}' for i in range(len(trainData_origin[0]))]
trainData = pd.DataFrame(data = trainData_origin,columns=colu)

In [28]:
# mr_MID = pymrmr.mRMR(trainData,'MID',246)
# mr_MID

In [27]:
#mr_MIQ = pymrmr.mRMR(trainData,'MIQ',246)
#mr_MIQ

In [23]:
pd.DataFrame(   np.hstack( (np.array(["sec_MRMR_MID","sec_MRMR_MIQ"]).reshape(-1,1),np.append( np.array(mr_MID).reshape(1,-1),np.array(mr_MIQ).reshape(1,-1),axis=0)))).to_csv("Result/Secondary_MRMR.csv")

In [ ]:
np.array(mr_MID).shape

In [30]:
from sklearn.decomposition import KernelPCA
scikit_kpca = KernelPCA(n_components=100, kernel='linear', gamma=3)
scikit_kpca.fit(trainData_origin)
trainData = scikit_kpca.transform(trainData_origin)

In [31]:
trainData.shape

(9126, 100)